In [16]:
import pandas as pd
import numpy as np
import math as m


df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header = None, names = [x for x in range(-1,100)])
del df[-1]
df.replace(99, np.NaN, True)
#df = df.loc[0:11000,:]
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(24983, 100)

In [4]:
# rescale between 0 and 1

df = (df + 10)/20

In [5]:
best_ratings = []

for i in df.columns:

  is_rating = df.loc[:,i] != np.NaN
  
  ratings = df.loc[is_rating, i]
  avg = np.mean(ratings)
  
  best_ratings.append(avg)


best_ratings.index(max(best_ratings)), best_ratings.index(min(best_ratings))

round(min(best_ratings),2)

0.31

In [6]:
def validation_set(data):
  df1 = data.copy()
  indices = []
  for e in data.columns:
    arr = data[e].values
    # break
    rated = np.where(arr!= np.NaN)
    choice = np.random.choice(len(rated[0]), size = int(len(rated[0])*0.1), replace = False)
    choice = choice.tolist()
    df1.loc[rated[0][choice], e] = 99

    indices.append(rated[0][choice])
  return df1, indices

df1, indices = validation_set(df)

In [7]:
user_ratings = df1.values

n_latent_factors = 2

latent_user_preferences = np.random.rand(user_ratings.shape[0], n_latent_factors)
latent_item_features = np.random.rand(user_ratings.shape[1], n_latent_factors)

latent_user_preferences


array([[0.16281239, 0.8254279 ],
       [0.67697488, 0.26084589],
       [0.20493596, 0.54324632],
       ...,
       [0.68805639, 0.26379206],
       [0.94213058, 0.20704088],
       [0.12849939, 0.92517792]])

In [8]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating) and not rating == 99: # ignore both NaN and 99
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 4) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [9]:
num_iter = 10
hist = sgd(num_iter)

Iteration 0/10:	MSE=0.132513
Iteration 4/10:	MSE=0.104822
Iteration 8/10:	MSE=0.102232


In [10]:
predictions = latent_user_preferences.dot(latent_item_features.T)

predictions

array([[0.48282053, 0.45352639, 0.387917  , ..., 0.5196588 , 0.41197726,
        0.50646432],
       [0.43312067, 0.40337893, 0.45436297, ..., 0.40000866, 0.41342597,
        0.44690577],
       [0.37306193, 0.34982868, 0.31811698, ..., 0.39009216, 0.32590268,
        0.39004764],
       ...,
       [0.43245809, 0.4027321 , 0.45458202, ..., 0.39882823, 0.41317039,
        0.4461583 ],
       [0.51368477, 0.47758205, 0.56433625, ..., 0.45858075, 0.50082229,
        0.5282571 ],
       [0.52604042, 0.49437839, 0.41482769, ..., 0.57103591, 0.44563415,
        0.55234609]])

In [11]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df1.columns

In [12]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"(99.0, 0.48282052623877186)","(0.9395, 0.4535263917228265)","(0.016999999999999994, 0.38791699536570357)","(0.092, 0.35999340633737237)","(0.12400000000000003, 0.4493981711101886)","(0.075, 0.47132973703117337)","(0.007500000000000018, 0.377176117617516)","(0.7085, 0.40521460191474856)","(0.050999999999999976, 0.4003379489083406)","(0.262, 0.48491866582116544)",...,"(0.641, 0.4793432997016464)","(nan, 0.43537290757607183)","(nan, 0.44954583006017507)","(nan, 0.43548084459719866)","(99.0, 0.5179950656841927)","(nan, 0.5022953742863633)","(0.2185, 0.48510864236996554)","(nan, 0.5196587952586191)","(nan, 0.41197725930395585)","(nan, 0.5064643192117226)"
1,"(0.704, 0.433120667311233)","(0.48550000000000004, 0.4033789311813213)","(0.818, 0.45436296787147307)","(99.0, 0.35276896594289503)","(0.381, 0.4174253222032275)","(0.016999999999999994, 0.48397384391567333)","(0.46349999999999997, 0.41035741590493646)","(0.233, 0.37671617660754886)","(0.9440000000000002, 0.38229392798175416)","(0.961, 0.4532449042436797)",...,"(0.641, 0.5104319048644146)","(99.0, 0.4755767703586878)","(0.48550000000000004, 0.5563411901369348)","(0.893, 0.47607336195166666)","(0.49050000000000005, 0.4194806359215524)","(0.39299999999999996, 0.4554542832600024)","(0.653, 0.4788845892880166)","(0.517, 0.40000866139787417)","(0.284, 0.41342596888680133)","(0.5535, 0.44690576751782557)"
2,"(nan, 0.37306192984927294)","(nan, 0.3498286781916959)","(nan, 0.31811697703367137)","(nan, 0.28331250533147256)","(0.9515, 0.3497064677051232)","(0.9635, 0.37475336888029753)","(0.9515, 0.30387782301082295)","(0.9635, 0.3153815019981141)","(nan, 0.3133334550825829)","(nan, 0.37783575289206806)",...,"(nan, 0.38427541750384947)","(nan, 0.35109382584616633)","(nan, 0.3738055209649762)","(0.954, 0.35124631438250276)","(nan, 0.39242978215680957)","(nan, 0.3889501150690917)","(99.0, 0.3823842164015128)","(nan, 0.3900921616894985)","(nan, 0.32590267757806)","(nan, 0.39004763762783395)"
3,"(99.0, 0.24182317743918874)","(0.9175000000000001, 0.22602301042511047)","(nan, 0.22894133972412128)","(nan, 0.19002203338411888)","(99.0, 0.22973713175822846)","(0.908, 0.2559905372635605)","(0.359, 0.21236596058484203)","(0.8105, 0.2072578215680006)","(nan, 0.20805721730419557)","(0.592, 0.24881619949859593)",...,"(nan, 0.26628096621763087)","(nan, 0.2457530154992049)","(nan, 0.2750182824882862)","(0.5265, 0.24593729552146462)","(nan, 0.24471912844693852)","(nan, 0.25316146985129745)","(nan, 0.2572076876582592)","(nan, 0.23872347456660925)","(nan, 0.2206266441938007)","(nan, 0.25124671893943323)"
4,"(99.0, 0.37344643764071245)","(99.0, 0.34711284815258064)","(0.2915, 0.41294475156773164)","(0.2305, 0.31010580887901285)","(0.568, 0.3627584405942595)","(0.58, 0.4294721643699807)","(0.852, 0.3681580395427422)","(0.7304999999999999, 0.32744429357275157)","(0.47800000000000004, 0.33423537380608653)","(0.7865, 0.3944305386160138)",...,"(0.7595000000000001, 0.4561219998610594)","(0.779, 0.42698293782223373)","(0.7135, 0.5101706685265983)","(0.7595000000000001, 0.4274907152438869)","(0.7865, 0.3526862851447068)","(0.5775, 0.3936714394408892)","(0.6555, 0.42160932832238923)","(0.8275, 0.3317224757024676)","(0.5900000000000001, 0.3651982735061194)","(0.58, 0.3838537520989085)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,"(0.522, 0.568491729439843)","(0.8714999999999999, 0.5324884694884446)","(0.954, 0.5031693895191598)","(0.6165, 0.43688840497047793)","(0.6599999999999999, 0.5353733537798608)","(0.8375, 0.5816515694478857)","(0.06050000000000004, 0.4755241237316521)","(0.47350000000000003, 0.482881362480543)","(0.06299999999999999, 0.48148244450763217)","(0.8615, 0.5789226494647819)",...,"(0.9414999999999999, 0.5994958394454881)","(0.43949999999999995, 0.5497259781254313)","(0.961, 0.5961096158773648)","(0.16499999999999998, 0.5500275172230534)","(0.9225, 0.5901858906070434)","(99.0, 0.5935444872687706)

## Lab Quiz

In [23]:
np.mean(round(np.max(df),2)) # mean highest rated joke

9.518400000000002

In [56]:
df.iloc[23151,7] # user 23151 joke 7 rating

2.28

In [53]:
movies = pd.read_excel('C:/Users/AD/Desktop/movies_latent_factors.xlsx', 'Items')
users = pd.read_excel('C:/Users/AD/Desktop/movies_latent_factors.xlsx', 'Users')
users.head(5)

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669


In [57]:
movies.head(5)

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566


In [59]:
round(movies.loc[movies.loc[:,'Movie ID'] == 557, 'Factor15'],3) # movie ID 557, factor15

42   -0.104
Name: Factor15, dtype: float64

In [68]:
movies.loc[:,['Movie ID','Factor2']].sort_values(by = ['Factor2'])  # highest loading of factor 2

,Movie ID,Factor2
16,122,-2.526300
15,121,-2.512933
14,120,-2.510061
51,604,-2.004450
79,1894,-1.711204
...,...,...
6,38,1.771799
32,275,1.777301
5,24,1.997440
59,680,2.085395


In [78]:
round(np.dot(users.loc[users.loc[:,'User'] == 4997, 'Factor1':'Factor15'].values[0],
      movies.loc[movies.loc[:,'Movie ID'] == 9331, 'Factor1':'Factor15'].values[0]),
      2)

-0.34

In [79]:
round(np.dot(users.loc[users.loc[:,'User'] == 1882, 'Factor1':'Factor15'].values[0],
      movies.loc[movies.loc[:,'Movie ID'] == 862, 'Factor1':'Factor15'].values[0]),
      2)

0.28

In [112]:
t = []

for i in range(0,100):
    v = np.dot(users.loc[users.loc[:,'User'] == 2067, 'Factor1':'Factor15'].values[0],
          movies.loc[i, 'Factor1':'Factor15'])
    t.append(tuple([v, movies.loc[i, 'Movie ID']]))

d = pd.DataFrame(t)
d.sort_values(by = [0]), d.head(5), movies.loc[(movies.loc[:,'Movie ID'] == 13) | (movies.loc[:,'Movie ID'] == 98) , 'Title']

(           0     1
 58 -2.000206   672
 57 -1.982124   671
 14 -1.959577   120
 15 -1.882861   121
 16 -1.825707   122
 ..       ...   ...
 26  0.663749   197
 91  0.696350  8587
 33  0.743205   278
 10  0.817016    98
 2   0.828942    13
 
 [100 rows x 2 columns],
           0   1
 0 -0.239497  11
 1  0.321743  12
 2  0.828942  13
 3  0.267385  14
 4 -0.406868  22,
 2     Forrest Gump (1994)
 10       Gladiator (2000)
 Name: Title, dtype: object)

In [114]:
t = []

for i in range(0,100):
    v = np.dot(users.loc[users.loc[:,'User'] == 5323, 'Factor1':'Factor15'].values[0],
          movies.loc[i, 'Factor1':'Factor15'])
    t.append(tuple([v, movies.loc[i, 'Movie ID']]))

d = pd.DataFrame(t)
d.sort_values(by = [0]), d.head(5), movies.loc[(movies.loc[:,'Movie ID'] == 155) | (movies.loc[:,'Movie ID'] == 238) |
                                                                                  (movies.loc[:,'Movie ID'] == 8587) , 'Title']

(           0     1
 90 -0.517215  8467
 51 -0.490145   604
 20 -0.466908   153
 56 -0.450451   664
 79 -0.421808  1894
 ..       ...   ...
 4   0.502264    22
 10  0.543291    98
 91  0.667856  8587
 27  0.694714   238
 21  0.759871   155
 
 [100 rows x 2 columns],
           0   1
 0  0.214068  11
 1  0.154537  12
 2  0.261073  13
 3 -0.187174  14
 4  0.502264  22,
 21    The Dark Knight (2008)
 27      The Godfather (1972)
 91      The Lion King (1994)
 Name: Title, dtype: object)